# Neural Cook

_by [FlorentF9](http://github.com/FlorentF9)_

In [1]:
### IMPORTS ###
import numpy as np
import tensorflow as tf
from data_utils import Dataset
import pickle

## Load data

In [3]:
# ID
run = '140817'

# Load data
train = Dataset(file='data/recipes.txt')
with open('dictionary_{}.pickle'.format(run), 'wb') as f:
    pickle.dump(train.dictionary, f)

## Model

In [2]:
def RNN(x, weights, biases):
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights) + biases

In [4]:
n_symbols = train.n_symbols
n_steps = 10
n_hidden = 128

# tf Graph input
x = tf.placeholder('float', [None, n_steps, n_symbols])
y = tf.placeholder('float', [None, n_symbols])

# Define weights
weights = tf.get_variable('weights', initializer=tf.random_normal([n_hidden, n_symbols]))
biases = tf.get_variable('biases', initializer=tf.random_normal([n_symbols]))

# Define operations
prediction = RNN(x, weights, biases)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Add ops for Tensorboard
tf.summary.scalar('loss', loss)
tf.summary.scalar('accuracy', accuracy)
merged = tf.summary.merge_all()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

## Training

In [5]:
N_EPOCHS = 3
EVAL_FREQUENCY = 1000
batch_size = 32
train_size = len(train.data)
best_loss = np.inf

batch_gen = train.batch(batch_size, n_steps)

# Launch the graph
with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs/'+run, sess.graph)
    sess.run(tf.global_variables_initializer())
    step = 1
    # Keep training until reach max iterations
    while step * batch_size / train_size < N_EPOCHS:
        batch_x, batch_y = next(batch_gen)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % EVAL_FREQUENCY == 0:
            # Calculate batch loss and accuracy
            summary, loss_, acc_ = sess.run([merged, loss, accuracy], feed_dict={x: batch_x, y: batch_y})
            writer.add_summary(summary, step)
            print("Epoch " + str(step * batch_size / train_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss_) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc_))
            # Save weights
            if loss_ < best_loss:
                best_loss = loss_
                save_path = saver.save(sess, "models/model_{}.ckpt".format(run))
                print("Model saved in file: %s" % save_path)

        step += 1
    print("Optimization Finished!")

Epoch 0.04640828852032973, Minibatch Loss= 2.809701, Training Accuracy= 0.28125
Model saved in file: models/model_140817.ckpt
Epoch 0.09281657704065946, Minibatch Loss= 2.283351, Training Accuracy= 0.40625
Model saved in file: models/model_140817.ckpt
Epoch 0.1392248655609892, Minibatch Loss= 2.080940, Training Accuracy= 0.43750
Model saved in file: models/model_140817.ckpt
Epoch 0.18563315408131892, Minibatch Loss= 2.785281, Training Accuracy= 0.28125
Epoch 0.23204144260164866, Minibatch Loss= 1.979953, Training Accuracy= 0.62500
Model saved in file: models/model_140817.ckpt
Epoch 0.2784497311219784, Minibatch Loss= 2.132978, Training Accuracy= 0.40625
Epoch 0.3248580196423081, Minibatch Loss= 1.838617, Training Accuracy= 0.56250
Model saved in file: models/model_140817.ckpt
Epoch 0.37126630816263784, Minibatch Loss= 1.635211, Training Accuracy= 0.59375
Model saved in file: models/model_140817.ckpt
Epoch 0.4176745966829676, Minibatch Loss= 1.650619, Training Accuracy= 0.50000
Epoch 0.

## Sample

In [19]:
with tf.Session() as sess:
    saver.restore(sess, "models/model_{}.ckpt".format(run))
    prompt = "Enter {} characters: ".format(n_steps)
    characters = input(prompt)
    if len(characters) != n_steps:
        print('Please enter {} characters.'.format(n_steps))
    try:
        symbols_in_keys = [train.dictionary[characters[i]] for i in range(len(characters))]
        for i in range(300):
            symbols_onehot = np.zeros((1, n_steps, n_symbols))
            for i in range(n_steps):
                symbols_onehot[0, i, symbols_in_keys[i]] = 1.0
            onehot_pred = sess.run(prediction, feed_dict={x: symbols_onehot})
            onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
            characters += train.reverse_dictionary[onehot_pred_index]
            symbols_in_keys = symbols_in_keys[1:]
            symbols_in_keys.append(onehot_pred_index)
        print(characters)
    except:
        print('Not in dictionary')

Enter 10 characters: Etape 2: F
Etape 2: Faire vercou en de le ter couper les poit encres de poitre fermit de vertir de vertir de vertir de vertir de vertir de vertir de vertir de vertir de vertir de vertir de vertir de vertir de vertir de vertir de vertir de vertir de vertir de vertir de vertir de vertir de vertir de vertir de vertir de ve
